# Iterators and generators
A generator allows us to get an element from a collection without first loading the full collection in memory. They are more relevant as the size of the collection or of the objects to be stored in it increases. 

In [1]:
def fibonacci_list(num_items):
    numbers = []
    a, b = 0, 1
    while len(numbers) < num_items:
        numbers.append(a)
        a, b = b, a+b
    return numbers

In [2]:
fibonacci_list(10)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [8]:
type(fibonacci_list(10))

list

In [3]:
def fibonacci_gen(num_items):
    a, b = 0, 1
    while num_items:
        yield a
        a, b = b, a+b
        num_items -= 1

In [7]:
for f in fibonacci_gen(10):
    print(f);

0
1
1
2
3
5
8
13
21
34


In [9]:
type(fibonacci_gen(10))

generator

## Memory usage and time complexity
Loading a list to be used for a loop operator more expensive than using a generator both in terms of memory and time. The trade-off is that we cannot reference an element in a generator and we cannot know the number of elements in it. We can do a test using a function to compute the number of fibonacci numbers that are divisible by three.

In [14]:
len([n for n in fibonacci_gen(100_000) if n % 3 == 0])

25000

In [15]:
len([n for n in fibonacci_list(100_000) if n % 3 == 0])

25000

In [11]:
%load_ext memory_profiler

In [12]:
%memit len([n for n in fibonacci_gen(100_000) if n % 3 == 0])

peak memory: 203.15 MiB, increment: 117.39 MiB


In [13]:
%memit len([n for n in fibonacci_list(100_000) if n % 3 == 0])

peak memory: 551.49 MiB, increment: 464.07 MiB


In [16]:
%timeit len([n for n in fibonacci_list(100_000) if n % 3 == 0])

2.31 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit len([n for n in fibonacci_gen(100_000) if n % 3 == 0])

1.97 s ± 94.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
